## Data Science Academy

## Deep Learning Book

### Capítulo 69 - Treinando Um Agente Baseado em IA Para Jogar Box no Atari com Linguagem Python

Este Jupyter Notebook é parte integrante do Deep Learning Book:
    
www.deeplearningbook.com.br   

Esta aula considera que você possui conhecimento em Linguagem Python. Se esse não for o caso, acesse nosso curso gratuito e comece aprender agora mesmo: <a href="https://www.datascienceacademy.com.br/course?courseid=python-fundamentos">Python Fundamentos Para Análise de Dados</a>.

Consideramos ainda que você já estudou os capítulos anteriores do Deep Learning Book.

Vamos desenvolver de forma simples (mas funcional) o algoritmo Q-learning em Python e compreender como ocorre o aprendizado do agente inteligente. E para isso vamos mandar nosso agente para a Gym.

O OpenAI Gym é um ToolKit open-source que facilita o desenvolvimento e comparação de algoritmos de Aprendizagem Por Reforço. Viste o site para compreender mais sobre o Gym: https://gym.openai.com/

O conteúdo deste Jupyter Notebook pode ser usado livremente e pedimos apenas a gentileza de citar a fonte.

Para executar no Google Colab, clique no menu superior em Runtime - Run all

No mesmo menu, em Change Runtime Type, você pode alterar a execução para GPU ou TPU.

In [1]:
# Instala o pacote gym
!pip install -q gym

In [2]:
# Instala a extensão para atari
!pip install -q gym[atari]

In [3]:
# Imports
import csv
import gym
import random
import numpy as np

## Definição do Problema

Conforme ensinamos em <a href="https://www.datascienceacademy.com.br/pages/todos-os-cursos-dsa">todos os cursos da DSA</a>, qualquer projeto de Ciência de Dados, Machine Learning ou IA deve começar com a definição clara do problema a ser resolvido, para que então possamos escolher as melhores ferramentas, técnicas, procedimentos e métodos de avaliação. Como diz aquele velho ditado: "Se você não sabe para onde vai, qualquer caminho serve". Para escolher o caminho ideal, primeiro temos que definir para onde estamos indo.

Nesta aula vamos desenvolver um agente inteligente usando o algoritmo Q-Learning. O método de aprendizado será a Aprendizagem Por Reforço que estudamos nos capítulos anteriores do curso. Nosso objetivo será Treinar Um Agente Baseado em IA Para Jogar Box no Atari (se você é mais jovem, pode não saber o que é o Atari. Ele marcou toda uma geração e podemos considerar como sendo o avô dos consoles de video game atuais. Mais detalhes aqui: https://en.wikipedia.org/wiki/Atari_2600).

Mas desenvolver um ambiente para a partir do zero para treinar o agente seria extremamente trabalhoso, concorda? Teríamos que praticamente desenvolver um jogo do Atari. É aqui onde o Gym mostra seu valor.

A equipe do Gym oferece ambientes prontos que você carrega em seu código e usa para treinar seu agente. Assim, podemos concentrar nossos esforços no algoritmo do agente, sem ter a preocupação inicial com o desenvolvimento do ambiente (para aplicações comerciais, teremos que desenvolver o ambiente também).

Aqui você encontra a lista de ambientes disponíveis no Gym:

https://github.com/openai/gym/wiki/Table-of-environments

E aqui animações dos ambientes (se você jogou Atari na sua infância/juventude, vai bater o saudosismo...rsrs):

https://gym.openai.com/envs/#atari

Usaremos o ambiente do jogo Boxing do Atari, com o ambiente Boxing-ram-v0: https://gym.openai.com/envs/Boxing-ram-v0/

In [4]:
# Cria uma instância de um ambiente que facilita o desenvolvimento e teste do nosso agente
env = gym.make("Boxing-ram-v0")

## Algoritmo Q-Learning

Algoritmos de Machine Learning são representados como Classes em linguagens de Programação. De fato, a Programação Orientada a Objetos é a base de programação em Machine Learning (se quiser aprender mais sobre isso, visite nosso curso gratuito de Python Fundamentos, pois há um capítulo inteiro dedicado ao tema).

Vamos descrever o algoritmo e então criar uma Classe Python para o QLearning.

Q-learning é um método de aprendizado por reforço que tenta decidir a melhor ação a ser tomada com base em uma tabela de estado e ações, a **Q-table**.

O espaço de estado e o espaço de ação são discretos e finitos. 

A ideia é que uma ação $a$ seja executada no estado $s$, que leva a um novo estado $s'$ e retorna uma recompensa $r$. Isso é referido como uma tupla de experiência $\langle s,a,r,s' \rangle$.

A Q-table $Q[s,a]$ é inicializada aleatoriamente e depois atualizada iterativamente através da **Equação de Bellman**:

$$Q^{(i+1)}[s,a] = (1-\alpha)\,Q^{(i)}[s,a] + \alpha \left( r + \gamma \,Q^{(i)}\left[s,\text{argmax}_{a} \,Q^{(i)}[s,a]\right] \right).$$

Na equação acima, $i$ é o índice de iteração, $\alpha$ é a taxa de aprendizado, e $\gamma$ é a taxa de desconto para recompensas futuras.

A **política ótima**:
$$\pi_*(s) = \text{argmax}_{a} \,Q^{(i)}[s,a] $$

retorna a melhor ação a ser tomada em um determinado estado.

Mais detalhes sobre a Equação de Bellman aqui: https://pt.qwe.wiki/wiki/Bellman_equation

In [5]:
# Classe para o Algoritmo QLearning
class QLearn:
    
    # Método construtor
    # Esse método será executado quando a classe for instanciada e um novo objeto criado
    # Os atributos da classe serão então inicializados com os argumentos enviados ao instanciar a classe
    def __init__(self, actions, epsilon, alpha, gamma):
        
        # Dicionário para receber os valores de q
        self.q = {}
        
        # Constante de Exploração
        self.epsilon = epsilon  
        
        # Constante de Desconto
        self.alpha = alpha  
        
        # Taxa de Desconto
        self.gamma = gamma  
        
        # Ações
        self.actions = actions

    # Método para obter o valor de q
    # Com base em cada ação e estado, obtém o valor de q
    # Observe que self.q é um dos atributos iniciados no método construtor da classe
    def getQ(self, estado, acao):
        return self.q.get((estado, acao), 0.0)

    # Método para o aprendizado de q
    # Aqui implementamos a Equação de Bellman
    def aprendeQ(self, estado, acao, recompensa, valor):
        '''
        Algoritmo Q-learning:
            Q(s, a) += alpha * (reward(s,a) + max(Q(s') - Q(s,a))
        '''
        
        # Obtém o valor anterior de q
        antigo_q = self.q.get((estado, acao), None)
        
        # Se o valor estiver vazio, coletamos a recompensa
        # Se não, geramos um novo valor de q
        if antigo_q is None:
            self.q[(estado, acao)] = recompensa
        else:
            self.q[(estado, acao)] = antigo_q + self.alpha * (valor - antigo_q)

    # Método para escolha da ação
    def escolheAcao(self, estado, return_q = False):
        
        # Usando List Comprehension, obtemos os valores de q para cada ação e estado
        # Observe que self.actions foi iniciado no construtor da classe
        q = [self.getQ(estado, a) for a in self.actions]
        
        # Grava o maior valor de q
        maxQ = max(q)

        # Agora introduzimos aleatoriedade às decisões do agente
        if random.random() < self.epsilon:
            minQ = min(q);
            mag = max(abs(minQ), abs(maxQ))
            
            # Adiciona valores aleatórios a todas as ações, recalculando maxQ
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))]
            maxQ = max(q)

        # Conta os valores máximos
        count = q.count(maxQ)
        
        # Caso haja vários valores máximos de ação de estado selecionamos um aleatório entre eles
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            i = random.choice(best)
        else:
            i = q.index(maxQ)
        
        # E alimentamos a variável de ação
        action = self.actions[i]
        
        # Por fim, retornamos q e a ação ou apenas a ação
        if return_q:  
            return action, q
        return action

    # Este é o método de aprendizado para o treinamento do agente
    def aprendizado(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions])
        self.aprendeQ(state1, action1, reward, reward + self.gamma * maxqnew)

In [6]:
# Função para registrar em disco a recompensa em cada episódio
def csv_log(episodio, g):
    f = open(f'./log.csv', mode = 'a+')
    writer = csv.writer(f)
    writer.writerow([episodio, g])
    f.close()

In [7]:
# Função para criar o estado a partir do ambiente do Gym
def cria_estado(atributos):
    return int("".join(map(lambda atributo: str(int(atributo)), atributos)))

In [8]:
# Função para o aprendizado de um episódio
def aprende_um_episodio(Q, episodio):
    
    # Variável de controle do loop de aprendizado
    done = False
    
    # Inicializa variáveis
    G, reward = 0, 0
    
    # Reset do ambiente
    state = env.reset()
    
    # Loop
    while not done:
        
        # Escolhe uma ação no estado atual
        action = Q.escolheAcao(cria_estado(state))
        
        # Cada ação gera resultados e coletamos todos eles: estado, recompensa, se chegamos ao final e info
        state2, reward, done, info = env.step(action)
        
        # Usamos os resultados anteriores para o agente aprender se aquela ação foi boa ou não
        Q.aprendizado(cria_estado(state), action, reward, cria_estado(state))
        
        # Vamos acumulando as recompensas
        G += reward
        
        # Para para o próximo estado
        state = state2
       
    # Ao fim do loop, gravamos o episódio em disco
    csv_log(episodio, G)
    
    # Imprimos o andamento do aprendizado
    print('Episódio {} Recompensa Total: {}'.format(episodio, G))

A função acima resume muito bem o que estudamos nos capítulos anteriores do Deep Learning Book sobre Aprendizado Por Reforço. 

In [9]:
# Cria o objeto Q (instância da Classe) inicializando os parâmetros para as primeiras escolhas do agente

# Hiperparâmetros iniciais
actions = list(range(0, 18))
epsilon = 0.4
alpha = 0.618
gamma = 0.9

# Instância da classe
Q = QLearn(actions, epsilon, alpha, gamma)

o treinamento leva bastante tempo e faremos várias rodadas de treinamento. Portanto, pratique sua paciência.

In [10]:
# Vamos para a primeira rodada de treinamento com 1000 episódios
# Aqui, o agente vai jogar Box, tomando suas decisões de forma aleatória e aprendendo com elas
# Cada recompensa faz com o que o agente aprenda a ação que gera o resultado, ou seja, vencer o jogo
for episodio in range(1, 1001):
    aprende_um_episodio(Q, episodio)

Episódio 1 Recompensa Total: 1.0
Episódio 2 Recompensa Total: -4.0
Episódio 3 Recompensa Total: 1.0
Episódio 4 Recompensa Total: 5.0
Episódio 5 Recompensa Total: 2.0
Episódio 6 Recompensa Total: 1.0
Episódio 7 Recompensa Total: 8.0
Episódio 8 Recompensa Total: 3.0
Episódio 9 Recompensa Total: 1.0
Episódio 10 Recompensa Total: 7.0
Episódio 11 Recompensa Total: 2.0
Episódio 12 Recompensa Total: 0.0
Episódio 13 Recompensa Total: 13.0
Episódio 14 Recompensa Total: 1.0
Episódio 15 Recompensa Total: 8.0
Episódio 16 Recompensa Total: 2.0
Episódio 17 Recompensa Total: 1.0
Episódio 18 Recompensa Total: -3.0
Episódio 19 Recompensa Total: -8.0
Episódio 20 Recompensa Total: 0.0
Episódio 21 Recompensa Total: 0.0
Episódio 22 Recompensa Total: 4.0
Episódio 23 Recompensa Total: -2.0
Episódio 24 Recompensa Total: -1.0
Episódio 25 Recompensa Total: 0.0
Episódio 26 Recompensa Total: 2.0
Episódio 27 Recompensa Total: -1.0
Episódio 28 Recompensa Total: -9.0
Episódio 29 Recompensa Total: 1.0
Episódio 30 Rec

Episódio 236 Recompensa Total: 3.0
Episódio 237 Recompensa Total: -2.0
Episódio 238 Recompensa Total: 18.0
Episódio 239 Recompensa Total: 4.0
Episódio 240 Recompensa Total: 0.0
Episódio 241 Recompensa Total: 2.0
Episódio 242 Recompensa Total: 5.0
Episódio 243 Recompensa Total: -6.0
Episódio 244 Recompensa Total: 2.0
Episódio 245 Recompensa Total: -2.0
Episódio 246 Recompensa Total: 4.0
Episódio 247 Recompensa Total: 5.0
Episódio 248 Recompensa Total: 0.0
Episódio 249 Recompensa Total: 2.0
Episódio 250 Recompensa Total: -3.0
Episódio 251 Recompensa Total: 1.0
Episódio 252 Recompensa Total: 1.0
Episódio 253 Recompensa Total: -3.0
Episódio 254 Recompensa Total: 10.0
Episódio 255 Recompensa Total: 2.0
Episódio 256 Recompensa Total: -2.0
Episódio 257 Recompensa Total: 1.0
Episódio 258 Recompensa Total: -3.0
Episódio 259 Recompensa Total: 1.0
Episódio 260 Recompensa Total: -5.0
Episódio 261 Recompensa Total: -6.0
Episódio 262 Recompensa Total: -1.0
Episódio 263 Recompensa Total: -6.0
Episódi

Episódio 468 Recompensa Total: 1.0
Episódio 469 Recompensa Total: 1.0
Episódio 470 Recompensa Total: 7.0
Episódio 471 Recompensa Total: -1.0
Episódio 472 Recompensa Total: 1.0
Episódio 473 Recompensa Total: -2.0
Episódio 474 Recompensa Total: 2.0
Episódio 475 Recompensa Total: 1.0
Episódio 476 Recompensa Total: -2.0
Episódio 477 Recompensa Total: -6.0
Episódio 478 Recompensa Total: -1.0
Episódio 479 Recompensa Total: -6.0
Episódio 480 Recompensa Total: 1.0
Episódio 481 Recompensa Total: -1.0
Episódio 482 Recompensa Total: -3.0
Episódio 483 Recompensa Total: -6.0
Episódio 484 Recompensa Total: 15.0
Episódio 485 Recompensa Total: 3.0
Episódio 486 Recompensa Total: 6.0
Episódio 487 Recompensa Total: -1.0
Episódio 488 Recompensa Total: 3.0
Episódio 489 Recompensa Total: 7.0
Episódio 490 Recompensa Total: -5.0
Episódio 491 Recompensa Total: -1.0
Episódio 492 Recompensa Total: -5.0
Episódio 493 Recompensa Total: 24.0
Episódio 494 Recompensa Total: -3.0
Episódio 495 Recompensa Total: 7.0
Epis

Episódio 699 Recompensa Total: -4.0
Episódio 700 Recompensa Total: 6.0
Episódio 701 Recompensa Total: -7.0
Episódio 702 Recompensa Total: 1.0
Episódio 703 Recompensa Total: 0.0
Episódio 704 Recompensa Total: -1.0
Episódio 705 Recompensa Total: 4.0
Episódio 706 Recompensa Total: -3.0
Episódio 707 Recompensa Total: 6.0
Episódio 708 Recompensa Total: 0.0
Episódio 709 Recompensa Total: 1.0
Episódio 710 Recompensa Total: 2.0
Episódio 711 Recompensa Total: 0.0
Episódio 712 Recompensa Total: 5.0
Episódio 713 Recompensa Total: -1.0
Episódio 714 Recompensa Total: 3.0
Episódio 715 Recompensa Total: 2.0
Episódio 716 Recompensa Total: 3.0
Episódio 717 Recompensa Total: 7.0
Episódio 718 Recompensa Total: 6.0
Episódio 719 Recompensa Total: 19.0
Episódio 720 Recompensa Total: -3.0
Episódio 721 Recompensa Total: 6.0
Episódio 722 Recompensa Total: -3.0
Episódio 723 Recompensa Total: 11.0
Episódio 724 Recompensa Total: 5.0
Episódio 725 Recompensa Total: 1.0
Episódio 726 Recompensa Total: 1.0
Episódio 72

Episódio 931 Recompensa Total: 1.0
Episódio 932 Recompensa Total: 1.0
Episódio 933 Recompensa Total: 12.0
Episódio 934 Recompensa Total: -2.0
Episódio 935 Recompensa Total: -4.0
Episódio 936 Recompensa Total: -11.0
Episódio 937 Recompensa Total: -3.0
Episódio 938 Recompensa Total: 0.0
Episódio 939 Recompensa Total: -2.0
Episódio 940 Recompensa Total: 4.0
Episódio 941 Recompensa Total: 6.0
Episódio 942 Recompensa Total: 1.0
Episódio 943 Recompensa Total: 15.0
Episódio 944 Recompensa Total: -1.0
Episódio 945 Recompensa Total: 4.0
Episódio 946 Recompensa Total: 9.0
Episódio 947 Recompensa Total: 11.0
Episódio 948 Recompensa Total: 9.0
Episódio 949 Recompensa Total: 2.0
Episódio 950 Recompensa Total: 1.0
Episódio 951 Recompensa Total: -2.0
Episódio 952 Recompensa Total: -20.0
Episódio 953 Recompensa Total: -1.0
Episódio 954 Recompensa Total: -1.0
Episódio 955 Recompensa Total: 7.0
Episódio 956 Recompensa Total: 3.0
Episódio 957 Recompensa Total: 7.0
Episódio 958 Recompensa Total: 0.0
Episó

In [11]:
# Vamos alterar os hiperparâmetros e executar mais uma rodada de treinamento
Q.epsilon = 0.3
Q.alpha = 0.518
Q.gamma = 0.8

# Rodada de treinamento
for episodio in range(1, 1001):
    aprende_um_episodio(Q, episodio)

Episódio 1 Recompensa Total: 4.0
Episódio 2 Recompensa Total: 4.0
Episódio 3 Recompensa Total: 3.0
Episódio 4 Recompensa Total: 10.0
Episódio 5 Recompensa Total: 0.0
Episódio 6 Recompensa Total: 1.0
Episódio 7 Recompensa Total: -2.0
Episódio 8 Recompensa Total: 1.0
Episódio 9 Recompensa Total: -5.0
Episódio 10 Recompensa Total: 3.0
Episódio 11 Recompensa Total: -5.0
Episódio 12 Recompensa Total: 0.0
Episódio 13 Recompensa Total: 5.0
Episódio 14 Recompensa Total: 1.0
Episódio 15 Recompensa Total: -4.0
Episódio 16 Recompensa Total: 5.0
Episódio 17 Recompensa Total: 0.0
Episódio 18 Recompensa Total: 4.0
Episódio 19 Recompensa Total: -2.0
Episódio 20 Recompensa Total: -1.0
Episódio 21 Recompensa Total: -1.0
Episódio 22 Recompensa Total: -1.0
Episódio 23 Recompensa Total: -1.0
Episódio 24 Recompensa Total: 0.0
Episódio 25 Recompensa Total: 3.0
Episódio 26 Recompensa Total: -6.0
Episódio 27 Recompensa Total: 2.0
Episódio 28 Recompensa Total: 10.0
Episódio 29 Recompensa Total: 9.0
Episódio 30

Episódio 236 Recompensa Total: 1.0
Episódio 237 Recompensa Total: 1.0
Episódio 238 Recompensa Total: 2.0
Episódio 239 Recompensa Total: -13.0
Episódio 240 Recompensa Total: 1.0
Episódio 241 Recompensa Total: 5.0
Episódio 242 Recompensa Total: -1.0
Episódio 243 Recompensa Total: 0.0
Episódio 244 Recompensa Total: 0.0
Episódio 245 Recompensa Total: -2.0
Episódio 246 Recompensa Total: 5.0
Episódio 247 Recompensa Total: 2.0
Episódio 248 Recompensa Total: 0.0
Episódio 249 Recompensa Total: -1.0
Episódio 250 Recompensa Total: -1.0
Episódio 251 Recompensa Total: -1.0
Episódio 252 Recompensa Total: 7.0
Episódio 253 Recompensa Total: 0.0
Episódio 254 Recompensa Total: 1.0
Episódio 255 Recompensa Total: -3.0
Episódio 256 Recompensa Total: -9.0
Episódio 257 Recompensa Total: 3.0
Episódio 258 Recompensa Total: -2.0
Episódio 259 Recompensa Total: 0.0
Episódio 260 Recompensa Total: 6.0
Episódio 261 Recompensa Total: 4.0
Episódio 262 Recompensa Total: 0.0
Episódio 263 Recompensa Total: -1.0
Episódio 

Episódio 468 Recompensa Total: -2.0
Episódio 469 Recompensa Total: 2.0
Episódio 470 Recompensa Total: -8.0
Episódio 471 Recompensa Total: 7.0
Episódio 472 Recompensa Total: 1.0
Episódio 473 Recompensa Total: 5.0
Episódio 474 Recompensa Total: -5.0
Episódio 475 Recompensa Total: 1.0
Episódio 476 Recompensa Total: -11.0
Episódio 477 Recompensa Total: 1.0
Episódio 478 Recompensa Total: -2.0
Episódio 479 Recompensa Total: 0.0
Episódio 480 Recompensa Total: 0.0
Episódio 481 Recompensa Total: 7.0
Episódio 482 Recompensa Total: 0.0
Episódio 483 Recompensa Total: -2.0
Episódio 484 Recompensa Total: -7.0
Episódio 485 Recompensa Total: -3.0
Episódio 486 Recompensa Total: -3.0
Episódio 487 Recompensa Total: 5.0
Episódio 488 Recompensa Total: 5.0
Episódio 489 Recompensa Total: -1.0
Episódio 490 Recompensa Total: -15.0
Episódio 491 Recompensa Total: -5.0
Episódio 492 Recompensa Total: -14.0
Episódio 493 Recompensa Total: 0.0
Episódio 494 Recompensa Total: -5.0
Episódio 495 Recompensa Total: 4.0
Epi

Episódio 700 Recompensa Total: 3.0
Episódio 701 Recompensa Total: 2.0
Episódio 702 Recompensa Total: 10.0
Episódio 703 Recompensa Total: 2.0
Episódio 704 Recompensa Total: 0.0
Episódio 705 Recompensa Total: 3.0
Episódio 706 Recompensa Total: 4.0
Episódio 707 Recompensa Total: 6.0
Episódio 708 Recompensa Total: 7.0
Episódio 709 Recompensa Total: 3.0
Episódio 710 Recompensa Total: 2.0
Episódio 711 Recompensa Total: 1.0
Episódio 712 Recompensa Total: 2.0
Episódio 713 Recompensa Total: -3.0
Episódio 714 Recompensa Total: -2.0
Episódio 715 Recompensa Total: 3.0
Episódio 716 Recompensa Total: -1.0
Episódio 717 Recompensa Total: 0.0
Episódio 718 Recompensa Total: 4.0
Episódio 719 Recompensa Total: 3.0
Episódio 720 Recompensa Total: 0.0
Episódio 721 Recompensa Total: -1.0
Episódio 722 Recompensa Total: -1.0
Episódio 723 Recompensa Total: 3.0
Episódio 724 Recompensa Total: -2.0
Episódio 725 Recompensa Total: 5.0
Episódio 726 Recompensa Total: 0.0
Episódio 727 Recompensa Total: 0.0
Episódio 728 

Episódio 932 Recompensa Total: 1.0
Episódio 933 Recompensa Total: 3.0
Episódio 934 Recompensa Total: -2.0
Episódio 935 Recompensa Total: 3.0
Episódio 936 Recompensa Total: -3.0
Episódio 937 Recompensa Total: -3.0
Episódio 938 Recompensa Total: 5.0
Episódio 939 Recompensa Total: 9.0
Episódio 940 Recompensa Total: 2.0
Episódio 941 Recompensa Total: -1.0
Episódio 942 Recompensa Total: 2.0
Episódio 943 Recompensa Total: -1.0
Episódio 944 Recompensa Total: -1.0
Episódio 945 Recompensa Total: -8.0
Episódio 946 Recompensa Total: 0.0
Episódio 947 Recompensa Total: 5.0
Episódio 948 Recompensa Total: -1.0
Episódio 949 Recompensa Total: -1.0
Episódio 950 Recompensa Total: 16.0
Episódio 951 Recompensa Total: 1.0
Episódio 952 Recompensa Total: 12.0
Episódio 953 Recompensa Total: 11.0
Episódio 954 Recompensa Total: 2.0
Episódio 955 Recompensa Total: 5.0
Episódio 956 Recompensa Total: 1.0
Episódio 957 Recompensa Total: 1.0
Episódio 958 Recompensa Total: -3.0
Episódio 959 Recompensa Total: -5.0
Episód

In [12]:
# Humm, pelo resultados anteriores, acredito que reduzindo os hiperparâmetros, ajudaremos nosso agente. 
# Vamos ajudá-lo.
Q.epsilon = 0.2
Q.alpha = 0.418
Q.gamma = 0.7

# Rodada de treinamento
for episodio in range(1, 1001):
    aprende_um_episodio(Q, episodio)

Episódio 1 Recompensa Total: -7.0
Episódio 2 Recompensa Total: 3.0
Episódio 3 Recompensa Total: -5.0
Episódio 4 Recompensa Total: -9.0
Episódio 5 Recompensa Total: 2.0
Episódio 6 Recompensa Total: -1.0
Episódio 7 Recompensa Total: 1.0
Episódio 8 Recompensa Total: -8.0
Episódio 9 Recompensa Total: 2.0
Episódio 10 Recompensa Total: 0.0
Episódio 11 Recompensa Total: 2.0
Episódio 12 Recompensa Total: 1.0
Episódio 13 Recompensa Total: -3.0
Episódio 14 Recompensa Total: 12.0
Episódio 15 Recompensa Total: 7.0
Episódio 16 Recompensa Total: 5.0
Episódio 17 Recompensa Total: 3.0
Episódio 18 Recompensa Total: 0.0
Episódio 19 Recompensa Total: -5.0
Episódio 20 Recompensa Total: -6.0
Episódio 21 Recompensa Total: -7.0
Episódio 22 Recompensa Total: 3.0
Episódio 23 Recompensa Total: -1.0
Episódio 24 Recompensa Total: 0.0
Episódio 25 Recompensa Total: 1.0
Episódio 26 Recompensa Total: 6.0
Episódio 27 Recompensa Total: 1.0
Episódio 28 Recompensa Total: -5.0
Episódio 29 Recompensa Total: 7.0
Episódio 30

Episódio 236 Recompensa Total: 8.0
Episódio 237 Recompensa Total: 0.0
Episódio 238 Recompensa Total: -2.0
Episódio 239 Recompensa Total: 1.0
Episódio 240 Recompensa Total: 0.0
Episódio 241 Recompensa Total: 5.0
Episódio 242 Recompensa Total: 7.0
Episódio 243 Recompensa Total: 2.0
Episódio 244 Recompensa Total: 4.0
Episódio 245 Recompensa Total: -11.0
Episódio 246 Recompensa Total: -3.0
Episódio 247 Recompensa Total: 5.0
Episódio 248 Recompensa Total: 4.0
Episódio 249 Recompensa Total: 1.0
Episódio 250 Recompensa Total: -4.0
Episódio 251 Recompensa Total: 0.0
Episódio 252 Recompensa Total: 5.0
Episódio 253 Recompensa Total: 0.0
Episódio 254 Recompensa Total: -2.0
Episódio 255 Recompensa Total: 4.0
Episódio 256 Recompensa Total: 2.0
Episódio 257 Recompensa Total: 0.0
Episódio 258 Recompensa Total: 4.0
Episódio 259 Recompensa Total: -3.0
Episódio 260 Recompensa Total: 0.0
Episódio 261 Recompensa Total: 0.0
Episódio 262 Recompensa Total: 6.0
Episódio 263 Recompensa Total: 1.0
Episódio 264 

Episódio 468 Recompensa Total: 4.0
Episódio 469 Recompensa Total: 10.0
Episódio 470 Recompensa Total: 8.0
Episódio 471 Recompensa Total: 0.0
Episódio 472 Recompensa Total: 0.0
Episódio 473 Recompensa Total: 2.0
Episódio 474 Recompensa Total: -18.0
Episódio 475 Recompensa Total: -3.0
Episódio 476 Recompensa Total: 5.0
Episódio 477 Recompensa Total: -2.0
Episódio 478 Recompensa Total: -21.0
Episódio 479 Recompensa Total: -2.0
Episódio 480 Recompensa Total: 1.0
Episódio 481 Recompensa Total: 1.0
Episódio 482 Recompensa Total: -4.0
Episódio 483 Recompensa Total: 4.0
Episódio 484 Recompensa Total: -1.0
Episódio 485 Recompensa Total: 3.0
Episódio 486 Recompensa Total: 8.0
Episódio 487 Recompensa Total: -1.0
Episódio 488 Recompensa Total: 0.0
Episódio 489 Recompensa Total: -29.0
Episódio 490 Recompensa Total: 0.0
Episódio 491 Recompensa Total: 2.0
Episódio 492 Recompensa Total: 2.0
Episódio 493 Recompensa Total: 0.0
Episódio 494 Recompensa Total: 10.0
Episódio 495 Recompensa Total: 5.0
Episód

Episódio 699 Recompensa Total: 2.0
Episódio 700 Recompensa Total: 18.0
Episódio 701 Recompensa Total: 1.0
Episódio 702 Recompensa Total: -6.0
Episódio 703 Recompensa Total: 8.0
Episódio 704 Recompensa Total: -5.0
Episódio 705 Recompensa Total: 1.0
Episódio 706 Recompensa Total: 3.0
Episódio 707 Recompensa Total: -14.0
Episódio 708 Recompensa Total: 0.0
Episódio 709 Recompensa Total: -2.0
Episódio 710 Recompensa Total: 0.0
Episódio 711 Recompensa Total: 7.0
Episódio 712 Recompensa Total: 0.0
Episódio 713 Recompensa Total: 6.0
Episódio 714 Recompensa Total: 4.0
Episódio 715 Recompensa Total: -4.0
Episódio 716 Recompensa Total: 2.0
Episódio 717 Recompensa Total: 3.0
Episódio 718 Recompensa Total: -5.0
Episódio 719 Recompensa Total: -2.0
Episódio 720 Recompensa Total: 4.0
Episódio 721 Recompensa Total: 8.0
Episódio 722 Recompensa Total: 3.0
Episódio 723 Recompensa Total: -15.0
Episódio 724 Recompensa Total: 2.0
Episódio 725 Recompensa Total: 0.0
Episódio 726 Recompensa Total: -1.0
Episódio

Episódio 931 Recompensa Total: -12.0
Episódio 932 Recompensa Total: -3.0
Episódio 933 Recompensa Total: -1.0
Episódio 934 Recompensa Total: -6.0
Episódio 935 Recompensa Total: -5.0
Episódio 936 Recompensa Total: 8.0
Episódio 937 Recompensa Total: 4.0
Episódio 938 Recompensa Total: -3.0
Episódio 939 Recompensa Total: 2.0
Episódio 940 Recompensa Total: 1.0
Episódio 941 Recompensa Total: 2.0
Episódio 942 Recompensa Total: 2.0
Episódio 943 Recompensa Total: 1.0
Episódio 944 Recompensa Total: 2.0
Episódio 945 Recompensa Total: -1.0
Episódio 946 Recompensa Total: 0.0
Episódio 947 Recompensa Total: 2.0
Episódio 948 Recompensa Total: -1.0
Episódio 949 Recompensa Total: 1.0
Episódio 950 Recompensa Total: -2.0
Episódio 951 Recompensa Total: -7.0
Episódio 952 Recompensa Total: 0.0
Episódio 953 Recompensa Total: 14.0
Episódio 954 Recompensa Total: 9.0
Episódio 955 Recompensa Total: 2.0
Episódio 956 Recompensa Total: 1.0
Episódio 957 Recompensa Total: 4.0
Episódio 958 Recompensa Total: 6.0
Episódio

In [13]:
# Podemos reduzir ainda mais.
# Porém acredito que técnicas mais avançadas devam ser usadas para treinar o agente nesse ambiente.
Q.epsilon = 0.1
Q.alpha = 0.318
Q.gamma = 0.6

# Rodada de treinamento
for episodio in range(1, 1001):
    aprende_um_episodio(Q, episodio)

Episódio 1 Recompensa Total: -9.0
Episódio 2 Recompensa Total: -4.0
Episódio 3 Recompensa Total: -7.0
Episódio 4 Recompensa Total: 8.0
Episódio 5 Recompensa Total: 3.0
Episódio 6 Recompensa Total: 1.0
Episódio 7 Recompensa Total: -7.0
Episódio 8 Recompensa Total: -2.0
Episódio 9 Recompensa Total: 6.0
Episódio 10 Recompensa Total: 4.0
Episódio 11 Recompensa Total: 16.0
Episódio 12 Recompensa Total: 1.0
Episódio 13 Recompensa Total: 0.0
Episódio 14 Recompensa Total: 1.0
Episódio 15 Recompensa Total: 16.0
Episódio 16 Recompensa Total: 1.0
Episódio 17 Recompensa Total: -1.0
Episódio 18 Recompensa Total: -2.0
Episódio 19 Recompensa Total: 1.0
Episódio 20 Recompensa Total: -2.0
Episódio 21 Recompensa Total: 2.0
Episódio 22 Recompensa Total: 2.0
Episódio 23 Recompensa Total: 4.0
Episódio 24 Recompensa Total: 0.0
Episódio 25 Recompensa Total: -2.0
Episódio 26 Recompensa Total: -2.0
Episódio 27 Recompensa Total: 2.0
Episódio 28 Recompensa Total: -2.0
Episódio 29 Recompensa Total: -5.0
Episódio 

Episódio 235 Recompensa Total: 1.0
Episódio 236 Recompensa Total: 7.0
Episódio 237 Recompensa Total: -8.0
Episódio 238 Recompensa Total: 12.0
Episódio 239 Recompensa Total: 7.0
Episódio 240 Recompensa Total: 8.0
Episódio 241 Recompensa Total: -2.0
Episódio 242 Recompensa Total: 2.0
Episódio 243 Recompensa Total: 3.0
Episódio 244 Recompensa Total: 4.0
Episódio 245 Recompensa Total: 2.0
Episódio 246 Recompensa Total: 1.0
Episódio 247 Recompensa Total: 5.0
Episódio 248 Recompensa Total: -2.0
Episódio 249 Recompensa Total: 14.0
Episódio 250 Recompensa Total: 0.0
Episódio 251 Recompensa Total: 6.0
Episódio 252 Recompensa Total: 4.0
Episódio 253 Recompensa Total: 2.0
Episódio 254 Recompensa Total: 1.0
Episódio 255 Recompensa Total: 9.0
Episódio 256 Recompensa Total: 6.0
Episódio 257 Recompensa Total: 10.0
Episódio 258 Recompensa Total: -2.0
Episódio 259 Recompensa Total: 0.0
Episódio 260 Recompensa Total: 3.0
Episódio 261 Recompensa Total: -2.0
Episódio 262 Recompensa Total: 0.0
Episódio 263

Episódio 467 Recompensa Total: -8.0
Episódio 468 Recompensa Total: -8.0
Episódio 469 Recompensa Total: 7.0
Episódio 470 Recompensa Total: -2.0
Episódio 471 Recompensa Total: 11.0
Episódio 472 Recompensa Total: -7.0
Episódio 473 Recompensa Total: 0.0
Episódio 474 Recompensa Total: -8.0
Episódio 475 Recompensa Total: -6.0
Episódio 476 Recompensa Total: 1.0
Episódio 477 Recompensa Total: 2.0
Episódio 478 Recompensa Total: 1.0
Episódio 479 Recompensa Total: 7.0
Episódio 480 Recompensa Total: 6.0
Episódio 481 Recompensa Total: 5.0
Episódio 482 Recompensa Total: 4.0
Episódio 483 Recompensa Total: 12.0
Episódio 484 Recompensa Total: -5.0
Episódio 485 Recompensa Total: 8.0
Episódio 486 Recompensa Total: 5.0
Episódio 487 Recompensa Total: 11.0
Episódio 488 Recompensa Total: 1.0
Episódio 489 Recompensa Total: -9.0
Episódio 490 Recompensa Total: 4.0
Episódio 491 Recompensa Total: 0.0
Episódio 492 Recompensa Total: 2.0
Episódio 493 Recompensa Total: 0.0
Episódio 494 Recompensa Total: 2.0
Episódio 

Episódio 698 Recompensa Total: 13.0
Episódio 699 Recompensa Total: 2.0
Episódio 700 Recompensa Total: -1.0
Episódio 701 Recompensa Total: 0.0
Episódio 702 Recompensa Total: -6.0
Episódio 703 Recompensa Total: -1.0
Episódio 704 Recompensa Total: -6.0
Episódio 705 Recompensa Total: 0.0
Episódio 706 Recompensa Total: -4.0
Episódio 707 Recompensa Total: -10.0
Episódio 708 Recompensa Total: 5.0
Episódio 709 Recompensa Total: -1.0
Episódio 710 Recompensa Total: 5.0
Episódio 711 Recompensa Total: -4.0
Episódio 712 Recompensa Total: 15.0
Episódio 713 Recompensa Total: -9.0
Episódio 714 Recompensa Total: 5.0
Episódio 715 Recompensa Total: 9.0
Episódio 716 Recompensa Total: 6.0
Episódio 717 Recompensa Total: 7.0
Episódio 718 Recompensa Total: 0.0
Episódio 719 Recompensa Total: 0.0
Episódio 720 Recompensa Total: 1.0
Episódio 721 Recompensa Total: -4.0
Episódio 722 Recompensa Total: 2.0
Episódio 723 Recompensa Total: 3.0
Episódio 724 Recompensa Total: 1.0
Episódio 725 Recompensa Total: -1.0
Episód

Episódio 930 Recompensa Total: -3.0
Episódio 931 Recompensa Total: 3.0
Episódio 932 Recompensa Total: 7.0
Episódio 933 Recompensa Total: 3.0
Episódio 934 Recompensa Total: 1.0
Episódio 935 Recompensa Total: -8.0
Episódio 936 Recompensa Total: 7.0
Episódio 937 Recompensa Total: -2.0
Episódio 938 Recompensa Total: 0.0
Episódio 939 Recompensa Total: -4.0
Episódio 940 Recompensa Total: -22.0
Episódio 941 Recompensa Total: 1.0
Episódio 942 Recompensa Total: 2.0
Episódio 943 Recompensa Total: -1.0
Episódio 944 Recompensa Total: -5.0
Episódio 945 Recompensa Total: 0.0
Episódio 946 Recompensa Total: 10.0
Episódio 947 Recompensa Total: 11.0
Episódio 948 Recompensa Total: -8.0
Episódio 949 Recompensa Total: 11.0
Episódio 950 Recompensa Total: -1.0
Episódio 951 Recompensa Total: 1.0
Episódio 952 Recompensa Total: -6.0
Episódio 953 Recompensa Total: -3.0
Episódio 954 Recompensa Total: -4.0
Episódio 955 Recompensa Total: 0.0
Episódio 956 Recompensa Total: 4.0
Episódio 957 Recompensa Total: -1.0
Epi

E isso é a Aprendizagem Por Reforço. 

O agente aprende por tentativa e erro, recebendo recompensas pelos acertos e tentando otimizar suas ações em cada estado.

Este é um algoritmo simples, visando mostrar para você os conceitos fundamentais. Podemos incorporar diversas técnicas a esse processo para acelerar o aprendizado. Poderíamos por exemplo, incluir uma rede neural profunda (Deep Learning) no processo de aprendizado (o que é chamado Deep Q-Learning em Deep Reinforcement Learning).

Curiosamente, é o que fazemos fazer a partir do Capítulo 70 aqui do Deep Learning Book.

**Nota:**

Gostaríamos de agradecer todo o feedback positivo que temos recebido sobre o Deep Learning Book. São vários e-mails de pessoas parabenizando pelo trabalho, trazendo sugestões, apontando possíveis melhorias e solicitando autorização para citação em trabalhos de conclusão de curso ou mesmo teses acadêmicas. Muitos professores de cursos de graduação e pós-graduação recomendam este livro para seus alunos.

Quando iniciamos o Deep Learning Book recebemos críticas do tipo: "A DSA não tem capacidade técnica para isso" ou "se ninguém desenvolveu um livro sobre o tema em português, porque a DSA conseguiria?" 

Cerca de 70 capítulos depois aí está o resultado (serão 100 capítulos no livro). Apenas ignoramos as críticas perniciosas, injustas e levianas, dando atenção às críticas respeitosas e bem fundamentadas. Focamos então no trabalho árduo diário, pois acreditamos que conhecimento transforma vidas e é disso que o Brasil precisa. Erros no meio do caminho podem ser cometidos e assim como o agente baseado em IA, aprendemos com eles e seguimos em frente.

Durante a sua jornada você vai encontrar mais pessoas para tacar pedra, do que para ajudar. Mas se você acredita no trabalho que está fazendo e como pode ajudar outras pessoas, siga firme e seja um elemento de transformação positiva. Muitas pessoas não tem capacidade ou vontade de fazer algo, e acharão que você também não tem e vão criticá-lo(a) apenas por isso. 

Sendo assim, fica a dica para você: Trabalhe bastante, até que seu trabalho não possa mais ser ignorado!

Estude, aprenda e divirta-se! 

Algumas aplicações comerciais de IA de última geração usam as técnicas que você encontra no trabalho deste e dos próximos capítulos.

No Capítulo 70 terá algo ainda mais interessante.

Acesse nossas redes sociais:

<a href="https://www.linkedin.com/company/data-science-academy/">LinkedIn</a>

<a href="https://www.facebook.com/dsacademybr">Facebook</a>

<a href="https://twitter.com/dsacademybr">Twitter</a>

<a href="https://www.instagram.com/dsacademybr/">Instagram</a>

<a href="https://www.youtube.com/channel/UCyoi9yYaQRQ2F34wDLosr2A?reload=9">Youtube</a>

Aqui você encontra todos os Cursos e Formações DSA:

<a href="https://www.datascienceacademy.com.br/pages/todos-os-cursos-dsa">Catálogo de Cursos</a>

<a href="https://www.datascienceacademy.com.br/pages/cursos-gratuitos-1">Curso Gratuitos</a>

# Fim